In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Derivace
Vypočtěte derivaci funkce `sin(x)` v bodě `x=1` metodou dopředných diferencí
$$
\frac{{\rm d}f}{{\rm d}x}\approx {\rm D_{FD}}(f, h)(x) = \frac{f(x+h) - f(x)}{h}
$$
Porovnejte s analyticky vypočtenou derivací. Jaká je relativní chyba v závislosti na volbě $h$ (v rozsahu $10^{-16} - 10^0$, vykreslete velikost chyby v log-log grafu). Dokážete vysvětlit pozorované chování?

In [ ]:
def f(x): return np.sin(x)
def dfdx(x): return np.cos(x)

In [ ]:
# dopredne diference (forward differences)
def diff_FD(f, x, dx): return # ukol 1

In [ ]:
# vypoctete derivaci pro x0=1. v zavislosti na dx
x0 = 1.0
dx = np.logspace(-16, 0, 500)

In [ ]:
rel_err_FD = np.abs((diff_FD(f, x0, dx) - dfdx(x0))/dfdx(x0))
plt.loglog(dx, rel_err_FD)

plt.grid()
from matplotlib.ticker import LogLocator
plt.ylim([1e-13, 1e1])
plt.gca().xaxis.set_major_locator(LogLocator(base=100., numticks=20))

Porovnejte s výpočtem metodou centrovaných diferencí

In [ ]:
# centrovane diference (central differences)
def diff_CD(f, x, dx): return # ukol 2

In [ ]:
rel_err_CD = np.abs((diff_CD(f, x0, dx) - dfdx(x0))/dfdx(x0))
plt.loglog(dx, rel_err_FD)
plt.loglog(dx, rel_err_CD)

plt.grid()
from matplotlib.ticker import LogLocator
plt.ylim([1e-13, 1e1])
plt.gca().xaxis.set_major_locator(LogLocator(base=100., numticks=20))

## Integrace
Vypočtěte integrál funkce `4/(1+x**2)` na intervalu `[0,1]` obdélníkovou metodou s dělením na 100 intervalů, $n=100$:
$$ s
I^{\rm OP} = \sum_{i=0}^{n-1} f(x_i + \xi h)h
$$
$$
x_i = a + i\frac{b-a}{n}
$$
Ověřte, jak přesnost výpočtu závisí na parametru $\xi\in[0, 1]$.

In [ ]:
def quad_rectangle(f, a, b, N, pos=0.5):
    # ukol 3

In [ ]:
def f(x): return 4/(1+x**2)

In [ ]:
xis = np.linspace(0, 1, 11)
ints = np.array([quad_rectangle(f, 0, 1, 100, xi) for xi in xis])

In [ ]:
plt.plot(xis, np.abs(ints-np.pi))
plt.grid()

Optimální volba je $\xi=0.5$, dostáváme tak "midpoint rule" druhého řádu přesnosti

### Richardsonova extrapolace
Řekněme, že hledaný integrál $I$ aproximujeme numericky vypočítanou hodnotou $\hat I(h)$ s chybou $E(h)$:
$$\hat I(h)  = I + E(h).$$

Pokud lze chybu metody $E$ vyjádřit jako polynom v závislosti na velikosti integračního kroku
$$
E = A_1h^{p_1} + A_2h^{p_2} + A_3h^{p_3} + \ldots,
$$
můžeme eliminovat dominantní člen rozvoje
$$
\hat I^{(1)} = \frac{2^{p_1} \hat I(h) - \hat I(2h)}{2^{p_1} - 1} = I + A'_2h^{p_2} + A'_3h^{p_3} + \ldots
$$


Vypočtěte integrál z předchozí úlohy lichoběžníkovou metodou s dělením na 10 a 5 intervalů. Využijte tyto aproximace k redukci chyby Richardsonovou metodou. Porovnejte chyby.

In [ ]:
def quad_trapezoid(f, a, b, N):
    # ukol 4

In [ ]:
N = 10

In [ ]:
print(quad_trapezoid(f, 0, 1, N//2)-np.pi)
print(quad_trapezoid(f, 0, 1, 10)-np.pi)


In [ ]:
def richardson(f, N0, p):
    # ukol 5

In [ ]:
richardson(lambda N:quad_trapezoid(f, 0, 1, N), 10, 2) - np.pi

### Rombergova integrace
Opakovanou aplikací Richardsonovy extrapolace lze postupně eliminovat další členy chybového rozvoje. Pokud použijeme lichoběžníkovou metodu k výpočtu integrálu s dělením na $2^N$ intervalů, můžeme extrapolovat $N$-krát, čímž dostaneme metodou řádu $2(N+1)$.

In [ ]:
def romberg(f, a, b, N):
    dx = (b-a)
    a = [0.5*dx*(f(a) + f(b))]
    for i in range(1, N):
        dx *= 0.5
        x = (np.arange(0, 2**(i), 2) + 1)*dx
        a.append(a[-1]*0.5 + np.sum(f(x))*dx)

    b = [np.array(a)]
    for j in range(1, N):
        b.append((2**(2*j) * b[-1][1:] - b[-1][:-1])/(2**(2*j) - 1))
    
    #for l in b:
    #    print(l-np.pi)
    return b[-1][0]

In [ ]:
romberg(f, 0, 1, 5)

In [ ]:
N = np.arange(1, 9)
I = np.array([romberg(f, 0, 1, n) for n in N])
I_trapezoid = np.array([quad_trapezoid(f, 0, 1, 2**(n-1)) for n in N])
h = 1/2**(N)

In [ ]:
plt.semilogy(N, np.abs(I-np.pi), label="Romberg")
plt.semilogy(N, np.abs(I_trapezoid-np.pi), label="trapezoid")
plt.legend()
plt.ylabel("error")
plt.xlabel("romberg N")
plt.grid()

In [ ]:
romberg(f, 0, 1, 9)

In [ ]:
np.pi

In [ ]:
plt.loglog(h, np.abs(I-np.pi), label="Romberg")
plt.loglog(h, np.abs(I_trapezoid-np.pi), label="trapezoid")
plt.legend()
plt.ylabel("error")
plt.xlabel("h")
plt.grid()